# 추천사유 생성

## 결과물 형태

In [49]:
# Schema
from typing_extensions import Annotated
from pydantic import Field, BaseModel

class IDRelevance(BaseModel):
    relevant_id: Annotated[
        list[str],
        Field(
            ..., 
            description=(
                "데이터의 ID 목록"
            ), 
        )
    ]
    reason: Annotated[
        list[str],
        Field(
            ..., 
            description="각 ID가 선정된 이유를 설명하는 문자열 목록. relevant_id와 인덱스가 일치해야 합니다.",
        )
    ]

## 프롬프트

In [50]:
# Prompt
from langchain_core.prompts import PromptTemplate

reason_template = '''
당신은 데이터 과학자이며, 사용자가 특정 연구 주제에 맞는 논문과 데이터셋을 추천받을 때,
각 항목이 왜 선택되었는지를 이해하기 쉽게 설명해야 합니다.

[작성 원칙]
1) 각 ID마다 개별 추천 이유를 반드시 1개 작성하세요. 총평 금지.
2) 추천 이유는 주제와 데이터 항목의 연결점을 분명히 밝혀야 합니다.
3) 연결 근거는 다음 중 2개 이상 포함: 키워드/토픽 유사성, 방법론 또는 모델 일치, 도메인/응용 맥락 일치
4) 객관적 설명형 문체 사용, 과장·평가는 금지
5) 각 이유는 1~2문장, 문장 길이 60자 내외
6) relevant_id와 reason의 개수는 반드시 동일하며, 인덱스 i가 서로 대응해야 합니다.
7) JSON 외 텍스트 출력 금지, key는 "relevant_id", "reason"만 사용

[Self-check]
출력 전에 relevant_id 길이와 reason 길이가 동일한지 스스로 점검하고, 동일하지 않다면 이유 리스트를 ID 개수에 맞춰 조정하세요.
이유가 과도하게 일반적이라면, 입력 주제 또는 해당 항목의 제목/키워드에서 최소 1개 근거 용어를 직접 인용하세요.

[Input]
연구 주제: {title}
연구 설명: {description}
키워드: {keyword}

[Data]
데이터 목록:
{data}

[Output(JSON)]
{{
  "relevant_id": [],
  "reason": []
}}
'''


reason_prompt = PromptTemplate.from_template(reason_template)


## 예시 데이터

In [51]:
# title, description, keyword
import json

with open("../data/input_data.json", "r", encoding="utf-8") as f:
    input_data = json.load(f)

try:
    title = input_data['dataset_title_etc_main']
    description = input_data['dataset_expl_etc_main']
    keyword = input_data['dataset_expl_etc_main']
    input_id = input_data['svc_id']

except:
    items = input_data["MetaData"]["recordList"]["record"]["item"]
    title = next(i["#text"] for i in items if i["@metaCode"] == "Title")
    description = next(i["#text"] for i in items if i["@metaCode"] == "Abstract")
    keyword = next(i["#text"] for i in items if i["@metaCode"] == "Keyword")
    input_id = next(i["#text"] for i in items if i["@metaCode"] == "CN")
    

In [52]:
# data
import pandas as pd

df_article = pd.read_csv('../data/search_results_article.csv', encoding='UTF-8', low_memory=False)
df_data = pd.read_csv('../data/search_results_dataset.csv', encoding='UTF-8', low_memory=False)

cleaned_df_data = (
    df_data[
        ['svc_id', 'dataset_title_etc_main', 'dataset_expl_etc_main','dataset_pub_dt_pc', 'dataset_kywd_etc_main', 'dataset_creator_etc_main', 'dataset_lndgpg', 'query']
    ]
    .rename(
        columns={
            'svc_id': 'ID',
            'dataset_title_etc_main': 'title',
            'dataset_expl_etc_main': 'description',
            'dataset_pub_dt_pc': 'pubyear',
            'dataset_kywd_etc_main': 'keyword',
            'dataset_creator_etc_main': 'author',
            'dataset_lndgpg': 'URL',
        }
    )
)
cleaned_df_data['category'] = 'dataset'

cleaned_df_arti = (
    df_article[
        ['CN', 'Title', 'Abstract', 'Pubyear', 'Keyword', 'Author', 'ContentURL', 'query']
    ]
    .rename(
        columns={
            'CN': 'ID',
            'Title': 'title',
            'Abstract': 'description',
            'Pubyear': 'pubyear',
            'Keyword': 'keyword',
            'Author': 'author',
            'ContentURL': 'URL'
        }
    )
)
cleaned_df_arti['category'] = 'article'

df = pd.concat([cleaned_df_arti, cleaned_df_data], ignore_index=True)


In [53]:
# relevance_data
relevance_df = pd.read_csv('../data/relevance_results.csv', encoding='UTF-8', low_memory=False)

## 작동 방식

In [54]:
# Node
from langchain_openai import ChatOpenAI

relevant_ids = relevance_df['ID'].tolist()
filtered_df = df[df['ID'].isin(relevant_ids)]

prompt = reason_prompt.invoke(
    {
        'title': title, 
        'description': description,
        'keyword': keyword,
        'data': filtered_df[['ID', 'title', 'description', 'keyword']].to_dict(orient="records"),
    }
)

sllm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

structured_sllm = sllm.with_structured_output(IDRelevance)
res = structured_sllm.invoke(prompt)
    
tmp = pd.DataFrame({
    'ID': res.relevant_id,
    'reason': res.reason
})

relevance_df = pd.merge(
    relevance_df[['ID', 'relevance']],
    tmp,
    on='ID',
    how='left'
)

relevance_df.to_csv('../data/relevance_results.csv', index=False, encoding='utf-8')

display(relevance_df)

,ID,relevance,reason
0,DIKO0010027329,92.394930,"이 연구는 한국인의 3D 무릎 관절 모델을 구축하여, 한국인에 적합한 인공관절 개발..."
1,NART73608690,53.741560,"이 연구는 한국인의 무릎 관절 치수 파라미터를 측정하여, 한국인 맞춤형 인공관절 설..."
2,NART51664194,51.529076,"이 연구는 MRI 이미지를 기반으로 3D 무릎 관절 모델을 재구성하여, 한국인의 무..."
3,f1bd34e52434a57a01b9ab55a6670891,51.367590,"이 연구는 한국인 남녀의 무릎 관절 형상 및 치수 데이터를 제공하여, 인공관절 설계..."
4,NPAP13253595,50.311615,"이 연구는 한국인 사체의 무릎관절 형상 모델을 다루고 있어, 한국인의 무릎 관절 특..."
